In [50]:
from CoReDataLoader import  dataset,dataloader, maxlen

In [58]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchmetrics as metrics

In [59]:
metric = metrics.Accuracy(task = "multiclass",num_classes = len(dataset.eoss))

In [53]:
class classifier(nn.Module):
    def __init__(self,input_length = maxlen):
        super().__init__()
        self.inlayer = nn.Linear(input_length,4096)
        self.bnorm = nn.BatchNorm1d(4096)
        self.silu = nn.SiLU()
        self.inter1 = nn.Linear(4096,2056)
        self.inter2 = nn.Linear(2056,1024)
        self.inter3 = nn.Linear(1024,1024)
        self.output = nn.Linear(1024,len(dataset.eoss))
        self.softmax = nn.Softmax()

        
    def forward(self,inp):
        inp = inp.to(torch.float32)
        itn = self.inlayer(inp)
        itn = self.bnorm(itn)
        itn = self.silu(itn)
        itn = self.silu(self.inter1(itn))
        itn = self.silu(self.inter2(itn))
        itn = self.silu(self.inter3(itn))
        itn = self.silu(self.output(itn))
        return itn
net = classifier()

In [54]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(),lr = 1,betas = (0.9,0.999))
epochs = 100
net.to("cpu")

classifier(
  (inlayer): Linear(in_features=40817, out_features=4096, bias=True)
  (bnorm): BatchNorm1d(4096, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (silu): SiLU()
  (inter1): Linear(in_features=4096, out_features=2056, bias=True)
  (inter2): Linear(in_features=2056, out_features=1024, bias=True)
  (inter3): Linear(in_features=1024, out_features=1024, bias=True)
  (output): Linear(in_features=1024, out_features=19, bias=True)
  (softmax): Softmax(dim=None)
)

In [61]:
losses = []
correct = 0
for epoch in range(epochs):
    for batch, (ts,params) in enumerate(dataloader):
        optimizer.zero_grad()
        outputs = net(ts)
        # print(outputs.shape,params[:,0].to(torch.long).shape)
        loss = criterion(outputs,params[:,0].to(torch.long))
        loss.backward()
        optimizer.step()
        losses.append(loss.item())

        print(f"{epoch+1}/{epochs} {batch+1}/{len(dataloader)} loss = {loss.item()} accuracy = {metric(outputs,params[:,0])*100}%\n",end = "\r",flush = True)
        correct = 0
    print(f"Epoch finished: {epoch+1}\n")

1/100 1/148 loss = 4376348160.0 accuracy = 0.0
1/100 2/148 loss = 49495433216.0 accuracy = 0.0625
1/100 3/148 loss = 21648537600.0 accuracy = 0.125
1/100 4/148 loss = 13987555328.0 accuracy = 0.5
1/100 5/148 loss = 8971236352.0 accuracy = 0.1875
1/100 6/148 loss = 4399267328.0 accuracy = 0.75
1/100 7/148 loss = 1845633280.0 accuracy = 0.0
1/100 8/148 loss = 1872699264.0 accuracy = 0.3125
1/100 9/148 loss = 750020032.0 accuracy = 0.0625
1/100 10/148 loss = 1129878784.0 accuracy = 0.0625
1/100 11/148 loss = 49428832256.0 accuracy = 0.1875
1/100 12/148 loss = 2726099712.0 accuracy = 0.25
1/100 13/148 loss = 259686464.0 accuracy = 0.125
1/100 14/148 loss = 534744416.0 accuracy = 0.1875
1/100 15/148 loss = 215072208.0 accuracy = 0.125
1/100 16/148 loss = 334612512.0 accuracy = 0.1875
1/100 17/148 loss = 491064960.0 accuracy = 0.375
1/100 18/148 loss = 2960518656.0 accuracy = 0.125
1/100 19/148 loss = 373913984.0 accuracy = 0.4375
1/100 20/148 loss = 406469728.0 accuracy = 0.375
1/100 21/148

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
plt.plot(losses)
plt.show()